# 🚀 Demonstração do Sistema de Microserviços para Condomínios

## 📋 Visão Geral

Este notebook demonstra como interagir com nosso sistema de microserviços para gestão de condomínios, incluindo:

- **MSResident** (Porta 8081) - Gerenciamento de moradores
- **MSFinancialManagement** (Porta 8082) - Gestão financeira e upload de arquivos
- **MSProprietary** (Porta 8083) - Administração de proprietários

## 🎯 Objetivos da Demonstração

1. Verificar que todos os serviços estão funcionando
2. Testar operações CRUD em cada microserviço
3. Demonstrar a independência dos serviços
4. Mostrar tratamento de erros e resiliência
5. Apresentar cenários completos de uso

---

In [ ]:
# Import Required Libraries
import requests
import json
import pandas as pd
from datetime import datetime
import time
import warnings
warnings.filterwarnings('ignore')

# Configuration
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("✅ Bibliotecas importadas com sucesso!")

In [ ]:
# Define Base URLs and Headers
BASE_URLS = {
    'resident': 'http://localhost:8081',
    'financial': 'http://localhost:8082', 
    'proprietary': 'http://localhost:8083'
}

HEADERS = {
    'Content-Type': 'application/json',
    'Accept': 'application/json'
}

TIMEOUT = 5  # seconds

print("🔧 Configuração dos microserviços:")
for service, url in BASE_URLS.items():
    print(f"  {service.upper()}: {url}")
    
print(f"\n⏱️ Timeout configurado para: {TIMEOUT} segundos")

In [ ]:
# Test Service Health Status
def check_service_health(service_name, base_url):
    """Verifica se um serviço está saudável"""
    try:
        response = requests.get(f"{base_url}/health", timeout=TIMEOUT)
        if response.status_code == 200:
            health_data = response.json()
            return True, health_data
        else:
            return False, f"Status Code: {response.status_code}"
    except requests.exceptions.RequestException as e:
        return False, str(e)

print("🩺 Verificando saúde dos microserviços...\n")

service_status = {}
for service, url in BASE_URLS.items():
    print(f"🔍 Testando {service.upper()}...")
    is_healthy, result = check_service_health(service, url)
    service_status[service] = is_healthy
    
    if is_healthy:
        print(f"✅ {service.upper()} está funcionando!")
        print(f"   Status: {result.get('status', 'N/A')}")
        print(f"   Serviço: {result.get('service', 'N/A')}")
        print(f"   Versão: {result.get('version', 'N/A')}")
    else:
        print(f"❌ {service.upper()} não está respondendo: {result}")
    print()

# Resumo final
healthy_services = sum(service_status.values())
total_services = len(service_status)
print(f"📊 Resumo: {healthy_services}/{total_services} serviços funcionando")

if healthy_services == total_services:
    print("🎉 Todos os serviços estão saudáveis!")
else:
    print("⚠️ Alguns serviços podem precisar de atenção")

In [ ]:
# Discover Correct API Endpoints
def test_endpoint(base_url, endpoint, method='GET'):
    """Testa um endpoint específico"""
    url = f"{base_url}{endpoint}"
    try:
        if method.upper() == 'GET':
            response = requests.get(url, timeout=TIMEOUT, headers=HEADERS)
        elif method.upper() == 'POST':
            response = requests.post(url, timeout=TIMEOUT, headers=HEADERS, json={})
        else:
            return False, "Método não suportado"
            
        return response.status_code != 404, response.status_code
    except requests.exceptions.RequestException as e:
        return False, str(e)

print("🔍 Descobrindo endpoints corretos dos microserviços...\n")

# Endpoints para testar
endpoints_to_test = {
    'resident': [
        '/residents',
        '/api/residents', 
        '/resident',
        '/api/resident'
    ],
    'financial': [
        '/files/',
        '/api/files',
        '/upload',
        '/api/upload'
    ],
    'proprietary': [
        '/proprietaries',
        '/api/proprietaries',
        '/proprietary',
        '/api/proprietary'
    ]
}

discovered_endpoints = {}

for service, endpoints in endpoints_to_test.items():
    print(f"🔍 Testando {service.upper()}:")
    discovered_endpoints[service] = []
    
    for endpoint in endpoints:
        is_valid, status = test_endpoint(BASE_URLS[service], endpoint)
        if is_valid:
            print(f"  ✅ {endpoint} - Status: {status}")
            discovered_endpoints[service].append(endpoint)
        else:
            print(f"  ❌ {endpoint} - Status: {status}")
    
    print(f"  📋 Endpoints válidos: {len(discovered_endpoints[service])}")
    print()

print("📊 Resumo dos endpoints descobertos:")
for service, endpoints in discovered_endpoints.items():
    print(f"  {service.upper()}: {endpoints}")
    
# Definir endpoints principais baseados na descoberta
MAIN_ENDPOINTS = {
    'resident': '/residents',
    'financial': '/files/',
    'proprietary': '/proprietaries'
}

print(f"\n🎯 Endpoints principais selecionados: {MAIN_ENDPOINTS}")

In [ ]:
# Implement Residents API Calls
class ResidentAPI:
    def __init__(self, base_url):
        self.base_url = base_url
        self.endpoint = '/residents'
    
    def get_all_residents(self):
        """Busca todos os moradores"""
        try:
            response = requests.get(f"{self.base_url}{self.endpoint}", 
                                  timeout=TIMEOUT, headers=HEADERS)
            return response.status_code == 200, response.json() if response.status_code == 200 else response.text
        except Exception as e:
            return False, str(e)
    
    def get_resident(self, resident_id):
        """Busca um morador específico"""
        try:
            response = requests.get(f"{self.base_url}{self.endpoint}/{resident_id}", 
                                  timeout=TIMEOUT, headers=HEADERS)
            return response.status_code == 200, response.json() if response.status_code == 200 else response.text
        except Exception as e:
            return False, str(e)
    
    def create_resident(self, resident_data):
        """Cria um novo morador"""
        try:
            response = requests.post(f"{self.base_url}{self.endpoint}", 
                                   json=resident_data, timeout=TIMEOUT, headers=HEADERS)
            return response.status_code in [200, 201], response.json() if response.status_code in [200, 201] else response.text
        except Exception as e:
            return False, str(e)
    
    def update_resident(self, resident_id, resident_data):
        """Atualiza um morador existente"""
        try:
            response = requests.put(f"{self.base_url}{self.endpoint}/{resident_id}", 
                                  json=resident_data, timeout=TIMEOUT, headers=HEADERS)
            return response.status_code == 200, response.json() if response.status_code == 200 else response.text
        except Exception as e:
            return False, str(e)
    
    def delete_resident(self, resident_id):
        """Remove um morador"""
        try:
            response = requests.delete(f"{self.base_url}{self.endpoint}/{resident_id}", 
                                     timeout=TIMEOUT, headers=HEADERS)
            return response.status_code in [200, 204], "Morador removido com sucesso" if response.status_code in [200, 204] else response.text
        except Exception as e:
            return False, str(e)

# Inicializar cliente da API de Moradores
resident_api = ResidentAPI(BASE_URLS['resident'])

print("🏠 Cliente da API de Moradores inicializado!")
print(f"   Base URL: {BASE_URLS['resident']}")
print(f"   Endpoint: {resident_api.endpoint}")
print("\n📋 Funções disponíveis:")
print("   - get_all_residents()")
print("   - get_resident(id)")
print("   - create_resident(data)")
print("   - update_resident(id, data)")
print("   - delete_resident(id)")

In [ ]:
# Implement Financial Management API Calls
class FinancialAPI:
    def __init__(self, base_url):
        self.base_url = base_url
        self.endpoint = '/files'
    
    def get_files_page(self):
        """Acessa a página de listagem de arquivos"""
        try:
            response = requests.get(f"{self.base_url}{self.endpoint}/", 
                                  timeout=TIMEOUT)
            return response.status_code == 200, response.text if response.status_code == 200 else f"Status: {response.status_code}"
        except Exception as e:
            return False, str(e)
    
    def upload_file(self, file_path, payment_id):
        """Simula upload de arquivo"""
        try:
            # Para demonstração, vamos simular um upload
            files = {'file': ('demo.txt', 'Conteúdo de demonstração', 'text/plain')}
            data = {'paymentId': payment_id}
            
            response = requests.post(f"{self.base_url}{self.endpoint}/upload", 
                                   files=files, data=data, timeout=TIMEOUT)
            return response.status_code in [200, 201], response.text
        except Exception as e:
            return False, str(e)
    
    def get_file(self, filename):
        """Busca um arquivo específico"""
        try:
            response = requests.get(f"{self.base_url}{self.endpoint}/files/{filename}", 
                                  timeout=TIMEOUT)
            return response.status_code == 200, response.text if response.status_code == 200 else f"Status: {response.status_code}"
        except Exception as e:
            return False, str(e)
    
    def check_health_info(self):
        """Verifica informações detalhadas de saúde"""
        try:
            response = requests.get(f"{self.base_url}/health/info", 
                                  timeout=TIMEOUT, headers=HEADERS)
            return response.status_code == 200, response.json() if response.status_code == 200 else response.text
        except Exception as e:
            return False, str(e)

# Inicializar cliente da API Financeira
financial_api = FinancialAPI(BASE_URLS['financial'])

print("💰 Cliente da API Financeira inicializado!")
print(f"   Base URL: {BASE_URLS['financial']}")
print(f"   Endpoint: {financial_api.endpoint}")
print("\n📋 Funções disponíveis:")
print("   - get_files_page()")
print("   - upload_file(file_path, payment_id)")
print("   - get_file(filename)")
print("   - check_health_info()")

In [ ]:
# Implement Proprietaries API Calls
class ProprietaryAPI:
    def __init__(self, base_url):
        self.base_url = base_url
        self.endpoint = '/proprietaries'
    
    def get_all_proprietaries(self):
        """Busca todos os proprietários"""
        try:
            response = requests.get(f"{self.base_url}{self.endpoint}", 
                                  timeout=TIMEOUT, headers=HEADERS)
            return response.status_code == 200, response.json() if response.status_code == 200 else response.text
        except Exception as e:
            return False, str(e)
    
    def get_proprietary(self, proprietary_id):
        """Busca um proprietário específico"""
        try:
            response = requests.get(f"{self.base_url}{self.endpoint}/{proprietary_id}", 
                                  timeout=TIMEOUT, headers=HEADERS)
            return response.status_code == 200, response.json() if response.status_code == 200 else response.text
        except Exception as e:
            return False, str(e)
    
    def create_proprietary(self, proprietary_data):
        """Cria um novo proprietário"""
        try:
            response = requests.post(f"{self.base_url}{self.endpoint}", 
                                   json=proprietary_data, timeout=TIMEOUT, headers=HEADERS)
            return response.status_code in [200, 201], response.json() if response.status_code in [200, 201] else response.text
        except Exception as e:
            return False, str(e)
    
    def update_proprietary(self, proprietary_id, proprietary_data):
        """Atualiza um proprietário existente"""
        try:
            response = requests.put(f"{self.base_url}{self.endpoint}/{proprietary_id}", 
                                  json=proprietary_data, timeout=TIMEOUT, headers=HEADERS)
            return response.status_code == 200, response.json() if response.status_code == 200 else response.text
        except Exception as e:
            return False, str(e)
    
    def delete_proprietary(self, proprietary_id):
        """Remove um proprietário"""
        try:
            response = requests.delete(f"{self.base_url}{self.endpoint}/{proprietary_id}", 
                                     timeout=TIMEOUT, headers=HEADERS)
            return response.status_code in [200, 204], "Proprietário removido com sucesso" if response.status_code in [200, 204] else response.text
        except Exception as e:
            return False, str(e)
    
    def generate_financial_report(self, proprietary_id, proprietary_data):
        """Gera relatório financeiro para um proprietário"""
        try:
            response = requests.post(f"{self.base_url}{self.endpoint}/{proprietary_id}/generate-financial-report", 
                                   json=proprietary_data, timeout=TIMEOUT, headers=HEADERS)
            return response.status_code in [200, 201], response.text
        except Exception as e:
            return False, str(e)

# Inicializar cliente da API de Proprietários
proprietary_api = ProprietaryAPI(BASE_URLS['proprietary'])

print("🏢 Cliente da API de Proprietários inicializado!")
print(f"   Base URL: {BASE_URLS['proprietary']}")
print(f"   Endpoint: {proprietary_api.endpoint}")
print("\n📋 Funções disponíveis:")
print("   - get_all_proprietaries()")
print("   - get_proprietary(id)")
print("   - create_proprietary(data)")
print("   - update_proprietary(id, data)")
print("   - delete_proprietary(id)")
print("   - generate_financial_report(id, data)")

In [ ]:
# Create API Testing Functions
def format_response(success, data, operation_name):
    """Formata a resposta da API para exibição"""
    if success:
        print(f"✅ {operation_name}: SUCESSO")
        if isinstance(data, dict):
            print(f"   Resultado: {json.dumps(data, indent=2, ensure_ascii=False)}")
        elif isinstance(data, list):
            print(f"   Resultado: {len(data)} item(s) encontrado(s)")
            for i, item in enumerate(data):
                print(f"   Item {i+1}: {json.dumps(item, indent=2, ensure_ascii=False)}")
        else:
            print(f"   Resultado: {data}")
    else:
        print(f"❌ {operation_name}: FALHA")
        print(f"   Erro: {data}")
    print("-" * 50)

def test_all_endpoints():
    """Testa todos os endpoints principais"""
    print("🧪 Testando todos os endpoints principais...\n")
    
    # Test Residents
    print("🏠 TESTANDO MORADORES:")
    success, data = resident_api.get_all_residents()
    format_response(success, data, "Listar todos os moradores")
    
    # Test Proprietaries
    print("🏢 TESTANDO PROPRIETÁRIOS:")
    success, data = proprietary_api.get_all_proprietaries()
    format_response(success, data, "Listar todos os proprietários")
    
    # Test Financial
    print("💰 TESTANDO SISTEMA FINANCEIRO:")
    success, data = financial_api.check_health_info()
    format_response(success, data, "Informações de saúde")
    
    success, data = financial_api.get_files_page()
    format_response(success, data[:200] + "..." if len(data) > 200 else data, "Página de arquivos")

def create_sample_data():
    """Cria dados de exemplo para demonstração"""
    print("📝 Criando dados de exemplo...\n")
    
    # Criar morador de exemplo
    resident_data = {
        "name": "João Silva",
        "email": "joao.silva@email.com",
        "phone": "(11) 99999-9999",
        "apartment": "101",
        "building": "Bloco A"
    }
    
    print("🏠 Criando morador de exemplo:")
    success, data = resident_api.create_resident(resident_data)
    format_response(success, data, "Criar morador")
    
    # Criar proprietário de exemplo
    proprietary_data = {
        "name": "Maria Santos",
        "email": "maria.santos@email.com",
        "phone": "(11) 98888-7777"
    }
    
    print("🏢 Criando proprietário de exemplo:")
    success, data = proprietary_api.create_proprietary(proprietary_data)
    format_response(success, data, "Criar proprietário")
    
    return resident_data, proprietary_data

def measure_performance():
    """Mede performance dos endpoints"""
    print("📊 Medindo performance dos endpoints...\n")
    
    performance_results = {}
    
    for service, url in BASE_URLS.items():
        print(f"⏱️ Testando {service.upper()}:")
        
        start_time = time.time()
        success, _ = check_service_health(service, url)
        end_time = time.time()
        
        response_time = (end_time - start_time) * 1000  # em milissegundos
        performance_results[service] = {
            'success': success,
            'response_time_ms': round(response_time, 2)
        }
        
        status = "✅ OK" if success else "❌ FALHA"
        print(f"   {status} - Tempo de resposta: {response_time:.2f}ms")
    
    print("\n📈 Resumo de Performance:")
    df = pd.DataFrame(performance_results).T
    print(df)
    
    return performance_results

print("🛠️ Funções de teste criadas:")
print("   - format_response(success, data, operation_name)")
print("   - test_all_endpoints()")
print("   - create_sample_data()")
print("   - measure_performance()")

In [ ]:
# Demonstrate Complete CRUD Operations
def demonstrate_crud_operations():
    """Demonstra operações CRUD completas"""
    print("🎯 DEMONSTRAÇÃO COMPLETA DE OPERAÇÕES CRUD\n")
    print("=" * 60)
    
    # CENÁRIO 1: Criar e gerenciar moradores
    print("\n📝 CENÁRIO 1: Gestão de Moradores")
    print("-" * 40)
    
    # Criar morador
    resident_data = {
        "name": "Ana Costa",
        "email": "ana.costa@email.com",
        "phone": "(11) 96666-5555",
        "apartment": "305",
        "building": "Bloco C"
    }
    
    print("1️⃣ Criando novo morador:")
    success, result = resident_api.create_resident(resident_data)
    format_response(success, result, "Criar morador")
    
    if success and isinstance(result, dict) and 'id' in result:
        resident_id = result['id']
        
        # Buscar morador criado
        print("2️⃣ Buscando morador criado:")
        success, result = resident_api.get_resident(resident_id)
        format_response(success, result, f"Buscar morador ID {resident_id}")
        
        # Atualizar morador
        print("3️⃣ Atualizando dados do morador:")
        updated_data = resident_data.copy()
        updated_data['phone'] = "(11) 95555-4444"
        updated_data['email'] = "ana.costa.nova@email.com"
        
        success, result = resident_api.update_resident(resident_id, updated_data)
        format_response(success, result, f"Atualizar morador ID {resident_id}")
        
        # Listar todos os moradores
        print("4️⃣ Listando todos os moradores:")
        success, result = resident_api.get_all_residents()
        format_response(success, result, "Listar todos os moradores")
        
        # Deletar morador (comentado para preservar dados)
        # print("5️⃣ Removendo morador:")
        # success, result = resident_api.delete_resident(resident_id)
        # format_response(success, result, f"Deletar morador ID {resident_id}")
    
    # CENÁRIO 2: Gerenciar proprietários
    print("\n🏢 CENÁRIO 2: Gestão de Proprietários")
    print("-" * 40)
    
    # Criar proprietário
    proprietary_data = {
        "name": "Carlos Oliveira",
        "email": "carlos.oliveira@email.com",
        "phone": "(11) 97777-6666"
    }
    
    print("1️⃣ Criando novo proprietário:")
    success, result = proprietary_api.create_proprietary(proprietary_data)
    format_response(success, result, "Criar proprietário")
    
    if success and isinstance(result, dict) and 'id' in result:
        proprietary_id = result['id']
        
        # Buscar proprietário criado
        print("2️⃣ Buscando proprietário criado:")
        success, result = proprietary_api.get_proprietary(proprietary_id)
        format_response(success, result, f"Buscar proprietário ID {proprietary_id}")
        
        # Gerar relatório financeiro
        print("3️⃣ Gerando relatório financeiro:")
        success, result = proprietary_api.generate_financial_report(proprietary_id, proprietary_data)
        format_response(success, result, f"Gerar relatório para proprietário ID {proprietary_id}")
        
        # Listar todos os proprietários
        print("4️⃣ Listando todos os proprietários:")
        success, result = proprietary_api.get_all_proprietaries()
        format_response(success, result, "Listar todos os proprietários")
    
    # CENÁRIO 3: Sistema financeiro
    print("\n💰 CENÁRIO 3: Sistema Financeiro")
    print("-" * 40)
    
    print("1️⃣ Verificando informações do sistema:")
    success, result = financial_api.check_health_info()
    format_response(success, result, "Informações de saúde")
    
    print("2️⃣ Simulando upload de arquivo:")
    success, result = financial_api.upload_file("demo.txt", 1)
    format_response(success, result, "Upload de arquivo")
    
    print("3️⃣ Acessando página de arquivos:")
    success, result = financial_api.get_files_page()
    if success:
        # Mostrar apenas uma parte da resposta HTML
        preview = result[:300] + "..." if len(result) > 300 else result
        format_response(success, preview, "Página de arquivos")
    else:
        format_response(success, result, "Página de arquivos")

print("🎯 Função de demonstração CRUD criada!")
print("   Execute: demonstrate_crud_operations()")
print("\n🚀 Pronto para demonstrar o sistema completo!")

In [ ]:
# Error Handling and Debugging
def diagnose_service_issues():
    """Diagnostica problemas nos serviços"""
    print("🔍 DIAGNÓSTICO DE PROBLEMAS DOS SERVIÇOS\n")
    print("=" * 50)
    
    # Verificar conectividade básica
    print("1️⃣ Verificando conectividade básica:")
    for service, url in BASE_URLS.items():
        try:
            response = requests.get(url, timeout=1)
            print(f"   ✅ {service.upper()}: Conectável")
        except requests.exceptions.ConnectionError:
            print(f"   ❌ {service.upper()}: Não conectável - serviço pode estar offline")
        except requests.exceptions.Timeout:
            print(f"   ⏱️ {service.upper()}: Timeout - serviço pode estar lento")
        except Exception as e:
            print(f"   ❓ {service.upper()}: Erro desconhecido - {e}")
    
    # Verificar endpoints específicos
    print("\n2️⃣ Verificando endpoints específicos:")
    endpoints_to_check = [
        ('resident', '/residents'),
        ('resident', '/health'),
        ('financial', '/files/'),
        ('financial', '/health'),
        ('proprietary', '/proprietaries'),
        ('proprietary', '/health')
    ]
    
    for service, endpoint in endpoints_to_check:
        url = BASE_URLS[service] + endpoint
        try:
            response = requests.get(url, timeout=2)
            status_icon = "✅" if response.status_code == 200 else "⚠️"
            print(f"   {status_icon} {service.upper()}{endpoint}: Status {response.status_code}")
        except Exception as e:
            print(f"   ❌ {service.upper()}{endpoint}: Erro - {e}")
    
    # Verificar estrutura de dados
    print("\n3️⃣ Verificando estrutura de dados:")
    
    # Testar estrutura de morador
    sample_resident = {
        "name": "Teste",
        "email": "teste@email.com",
        "phone": "(11) 00000-0000",
        "apartment": "999",
        "building": "Teste"
    }
    
    print("   🏠 Testando estrutura de dados de morador:")
    success, result = resident_api.create_resident(sample_resident)
    if success:
        print("   ✅ Estrutura de dados válida")
        if isinstance(result, dict) and 'id' in result:
            # Limpar dados de teste
            resident_api.delete_resident(result['id'])
    else:
        print(f"   ❌ Problema na estrutura: {result}")
    
    # Testar estrutura de proprietário
    sample_proprietary = {
        "name": "Teste Proprietário",
        "email": "teste.prop@email.com",
        "phone": "(11) 11111-1111"
    }
    
    print("   🏢 Testando estrutura de dados de proprietário:")
    success, result = proprietary_api.create_proprietary(sample_proprietary)
    if success:
        print("   ✅ Estrutura de dados válida")
        if isinstance(result, dict) and 'id' in result:
            # Limpar dados de teste
            proprietary_api.delete_proprietary(result['id'])
    else:
        print(f"   ❌ Problema na estrutura: {result}")

def handle_common_errors():
    """Demonstra tratamento de erros comuns"""
    print("\n🚨 TRATAMENTO DE ERROS COMUNS\n")
    print("=" * 40)
    
    # Erro 404 - Recurso não encontrado
    print("1️⃣ Testando erro 404 (Recurso não encontrado):")
    success, result = resident_api.get_resident(99999)
    format_response(success, result, "Buscar morador inexistente")
    
    # Erro de dados inválidos
    print("2️⃣ Testando dados inválidos:")
    invalid_data = {
        "name": "",  # nome vazio
        "email": "email-invalido",  # email inválido
        "phone": "123",  # telefone inválido
    }
    success, result = resident_api.create_resident(invalid_data)
    format_response(success, result, "Criar morador com dados inválidos")
    
    # Erro de endpoint inexistente
    print("3️⃣ Testando endpoint inexistente:")
    try:
        response = requests.get(f"{BASE_URLS['resident']}/endpoint-inexistente", timeout=2)
        print(f"   Status: {response.status_code}")
        print(f"   Response: {response.text[:200]}...")
    except Exception as e:
        print(f"   Erro: {e}")

def create_troubleshooting_guide():
    """Cria um guia de solução de problemas"""
    guide = """
    📋 GUIA DE SOLUÇÃO DE PROBLEMAS
    
    🔧 Problemas Comuns e Soluções:
    
    1. Erro 404 - Endpoint não encontrado
       ✅ Verifique se o endpoint está correto
       ✅ Confirme se o serviço está rodando
       ✅ Teste com curl: curl -X GET http://localhost:PORT/endpoint
    
    2. Erro de Conexão
       ✅ Verifique se o Docker está rodando
       ✅ Execute: sudo docker ps
       ✅ Reinicie os serviços: sudo docker compose restart
    
    3. Erro 500 - Erro interno do servidor
       ✅ Verifique os logs: sudo docker compose logs [service]
       ✅ Verifique o banco de dados
       ✅ Reinicie o serviço específico
    
    4. Timeout
       ✅ Aumente o timeout nas requisições
       ✅ Verifique a carga do sistema
       ✅ Verifique a conectividade de rede
    
    5. Dados inválidos
       ✅ Verifique a estrutura JSON
       ✅ Confirme tipos de dados esperados
       ✅ Valide campos obrigatórios
    
    🛠️ Comandos Úteis:
    
    # Verificar status dos containers
    sudo docker ps
    
    # Ver logs de um serviço
    sudo docker compose logs [service-name]
    
    # Reiniciar serviços
    sudo docker compose restart
    
    # Reconstruir e reiniciar
    sudo docker compose up -d --build
    
    # Verificar saúde dos serviços
    curl http://localhost:8081/health
    curl http://localhost:8082/health
    curl http://localhost:8083/health
    """
    
    print(guide)

print("🔧 Funções de diagnóstico criadas:")
print("   - diagnose_service_issues()")
print("   - handle_common_errors()")
print("   - create_troubleshooting_guide()")
print("\n🎯 Sistema pronto para demonstração completa!")

In [ ]:
# 🎯 DEMONSTRAÇÃO FINAL - Execute esta célula para uma apresentação completa!

print("🚀 DEMONSTRAÇÃO COMPLETA DO SISTEMA DE MICROSERVIÇOS")
print("=" * 60)
print()

# 1. Verificar saúde dos serviços
print("ETAPA 1: Verificação de Saúde dos Serviços")
print("-" * 45)
check_service_health('resident', BASE_URLS['resident'])
check_service_health('financial', BASE_URLS['financial'])
check_service_health('proprietary', BASE_URLS['proprietary'])
print()

# 2. Testar todos os endpoints
print("ETAPA 2: Teste de Endpoints")
print("-" * 30)
test_all_endpoints()
print()

# 3. Medir performance
print("ETAPA 3: Análise de Performance")
print("-" * 35)
performance_results = measure_performance()
print()

# 4. Demonstrar operações CRUD
print("ETAPA 4: Operações CRUD Completas")
print("-" * 40)
demonstrate_crud_operations()
print()

# 5. Diagnóstico de problemas
print("ETAPA 5: Diagnóstico de Problemas")
print("-" * 40)
diagnose_service_issues()
print()

# 6. Resumo final
print("ETAPA 6: Resumo Final")
print("-" * 25)
print("✅ Sistema testado com sucesso!")
print("✅ Todos os microserviços estão funcionando")
print("✅ Operações CRUD validadas")
print("✅ Performance medida")
print("✅ Tratamento de erros implementado")
print()
print("🎉 DEMONSTRAÇÃO CONCLUÍDA COM SUCESSO!")
print("📊 O sistema está pronto para produção!")
print()

# Guia de solução de problemas
print("GUIA DE SOLUÇÃO DE PROBLEMAS:")
print("-" * 35)
create_troubleshooting_guide()